# TERA - Aula 30
## Redução de Dimensionalidade e Topic Analysis

Objetivos gerais de topic analysis:
- Agrupar dados de forma a criar representações sumarizadas (sumarização de dados)
- Redução de dimensionalidade
- Clusters encontrados são explicados (encontra tópicos)

# Índice

- [Exemplo inicial](#Exemplo-Inicial)
- [PCA](#Principal-Component-Analysis-(PCA)
- [T-SNE](#T-SNE)
- [Topic Analysis](#Topic-Analysis)
    - [NMF](#Non-Negative-Matrix-Factorization-(NMF)
    - [LDA](#Latent-Dirichlet-Allocation-(LDA)
- [Case Subcategorias Elo7](#Case-Elo7---Subcategorias-Automáticas)
- [Case Sistemas de Recomendação](#Case-Sistema-de-Recomendação)

In [ ]:
# Imports usados no curso
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

sns.set(style="ticks")
plt.rcParams['figure.figsize'] = (16.0, 10.0)
plt.style.use('seaborn-colorblind')

In [ ]:
# Pasta contendo os dados:
ROOT_FOLDER = os.path.realpath('..')
DATASET_FOLDER = os.path.join(ROOT_FOLDER,'datasets')

---
# Exemplo inicial
- **Case Elo7: Clustering de Frete**

Vamos voltar para o problema de agrupamento de rotas para melhoria do frete no Elo7. Nosso vetor de atributos tem dimensão 4, o que nos impossibilita de observá-lo em apenas um gráfico. Essa limitação é um problema tanto do ponto de vista de intuição sobre o problema, quanto da correta observação dos resultados.

In [ ]:
df_route = pd.read_csv(os.path.join(DATASET_FOLDER, 'route_clustering_elo7_dataset.csv'), sep=';')

df_route.head()

Para facilitar os cálculos de distância, as latitudes e longitudes dos locais já foram realizados.

Vamos observar a distribuição dos nossos dados. Como temos 4 dimensões, uma forma de fazer isso seria uma matriz de gráficos pareados.

In [ ]:
sns.pairplot(df_route[['latitude_origem','longitude_origem','latitude_destino','longitude_destino']])
plt.show()

Ok, 4 dimensões ainda é possível ter uma boa intuição sobre o problema. Mas, e mais do que 4?

### Exemplo 2
- **Variedades de grãos de trigo**

Vamos utilizar o dataset de diferentes tipos de grãos de trigo obtidos pelo [UCI](https://archive.ics.uci.edu/ml/datasets/seeds#).

<img src="https://kaggle2.blob.core.windows.net/datasets-images/904/1650/133a751e78dabf09031d6adfa0075ae0/dataset-original.jpg" width="400" height="100" />

Esse dataset contém medidas de raio-x de três variedades de grãos de trigo. Gostaríamos de 

O dataset contém os seguintes parâmetros:

- `area`: Área total do grão, A
- `perimeter`: Perímetro do grão, P
- `compactness`: Grão de compactação do grão - $C = \frac{4 \pi A}{P^2}$
- `length_kernel`: Comprimento do núcleo
- `width_kernel`: Largura do núcleo
- `asymmetry`: Coeficiente de assimetria
- `kernel_groove`: Comprimento do sulco do núcleo

Variedades de grãos: 'Kama' (1), 'Rosa' (2) e 'Canadian' (3)

Agora temos um dataset com 7 dimensões. As coisas estão ficando mais complicadas.

In [ ]:
# Importar os dados
df_grain = pd.read_csv(os.path.join(DATASET_FOLDER, 'seeds_dataset.csv'), sep=';')
df_grain.head(5)

Vamos aplicar agora a matriz de gráficos pareados para tentar visualizar o problema.

In [ ]:
sns.pairplot(df_grain, hue='varieties')
plt.show()

Estamos agora sofrendo um pouco mais para visualizar os dados. O número de gráficos cresce ao quadrado em relação ao número de dimensões!

---
### Exemplo 3
- **Case Elo7 - Motivos de Compra**

Percebemos até agora que quanto maior o número de dimensões, menor nossa capacidade de observar e gerar intuição sobre o problema. O que podemos fazer agora com um problema de NLP, onde normalmente temos milhares de dimensões (cada palavra do vocabulário representa uma dimensão)?

Vamos verificar no dataset de motivos de compra dos usuários do Elo7.

In [ ]:
df_reason = pd.read_csv(os.path.join(DATASET_FOLDER, 'purchase_reason_elo7_dataset.csv'), sep=';')

df_reason.head(10)

Vamos utilizar o Tf-Idf para criar o embedding dos motivos de compra.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.9, max_features=500, sublinear_tf=True, use_idf=True)

Cria a matriz de embeddings.

In [ ]:
X = tfidf.fit_transform(df_reason['reason'].values)

In [ ]:
X.shape

Podemos observar que a matriz de atributos agora tem 496 dimensões! Para representar essas dimensões em uma matriz de gráficos pareados, nós precisaríamos de $496^2=246016$ gráficos! Totalmente impraticável!

Mas, a questão que deveria ser levantada é: Será que todas essas dimensões são necessárias para representar **fielmente** o problema? Será que não conseguiríamos simplificar de alguma forma? Isso é, não seria possível remover ou modificar certas dimensões que sejam "menos importantes" sem remover muita informação?
- Resposta: Sim, é possível!

---
# Principal Component Analysis (PCA)

O [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) é uma das técnicas mais utilizadas em Machine Learning para encontrar representações sumarizadas sobre os problemas de alta dimensionalidade. O PCA procura descorrelacionar linearmente os atributos do sistema, encontrando assim os **componentes principais**. Cada componente pode ser visto como um novo vetor de atributos que é formado pela combinação linear dos atributos originais. Os componentes principais são ordenados de forma que o primeiro componente representa a região de maior variância do sistema, o segundo representa a segunda maior e é ortogonal ao primeiro, e assim sucessivamente.

Vamos aplicar o método PCA nos problemas anteriores para verificar se conseguimos visualizar melhor o problema. Também podemos tentar reduzir o número de dimensões do nosso problema.

O algoritmo básico do PCA realiza 4 passos fundamentais para transformar o espaço de atributos e achar os componentes princiais:

1. Remove a média amostral
2. Rotaciona os eixos para descorrelacionar os dados
3. Ordena os componentes principais por nível de variância (importância)
4. Elimina os componentes menos variantes (Opcional - redução de dimensionalidade)

A implementação do PCA no sklearn pode ser encontrada neste [link](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

#### Elo7: Clustering Frete

In [ ]:
df_route = pd.read_csv(os.path.join(DATASET_FOLDER, 'route_clustering_elo7_dataset.csv'), sep=';')

df_route.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_route[['latitude_origem','longitude_origem','latitude_destino','longitude_destino']].values

# Normaliza os dados
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
# Vamos agora importar o método PCA do scikit-learn
from sklearn.decomposition import PCA

# Criemos agora a instância PCA
# Não definimos o número de features: não realiza passo 4
pca = PCA()

In [ ]:
X_pca = pca.fit_transform(X_scaled)

In [ ]:
X_pca.shape

Observe que ainda temos 4 dimensões. O PCA não reduziu a dimensionalidade do problema. Esse passo pode ser realizado posteriormente. O que temos agora é uma relação de "importância" entre os componentes principais. Vamos plotar o gráfico com apenas os dois primeiros componentes para verificar o resultado.

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.8, edgecolor='k')
plt.xlabel('Primeiro componente principal')
plt.ylabel('Segundo componente principal')
plt.grid()
plt.show()

Vamos observar como ficariam os clusters encontrados na aula de Clustering.

In [ ]:
# Encontra os clusters da aula anterior
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
labels = kmeans.fit_predict(X_scaled)

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='rainbow', alpha=0.8, edgecolor='k')
plt.xlabel('Primeiro componente principal')
plt.ylabel('Segundo componente principal')
plt.grid()
plt.show()

O que achou? Esses dois eixos nos mostram bons detalhes sobre os clusters encontrados anteriormente.

Outra coisa que podemos fazer é verificar o quanto cada eixo representa da variância do modelo, ou também chamado de **variância explicada** pelo componente.

In [ ]:
# Componentes principais
features = range(pca.n_components_)

plt.bar(features, pca.explained_variance_ratio_)
plt.xticks(features)
plt.xlabel('Componentes principais')
plt.ylabel('Variância')
plt.show()

print('Variância explicada: ')
cum_exp_var = np.cumsum(pca.explained_variance_ratio_)

print('-'*50)
for i in range(len(pca.explained_variance_ratio_)):
    print('{} Componente(s) principal(is): {:.2%}'.format(i+1, cum_exp_var[i]))

Podemos perceber que os dois primeiros eixos em conjunto representam cerca de 63% da variância explicada. Isso ainda é pouco, mas já foi possível verificar uma boa representação dos dados. Outro fato a ser observado é que o quarto eixo não possui grande influência no sistema. Se retirássemos ele, ainda teríamos 3 eixos que explicariam 82% da variância do sistema. Se não estivermos preocupados com uma grande precisão, poderíamos retirar esse componente.

#### Variedade de grãos

Vamos praticar para um problema com mais dimensões.

In [ ]:
# Importar os dados
df_grain = pd.read_csv(os.path.join(DATASET_FOLDER, 'seeds_dataset.csv'), sep=';')
df_grain.head(5)

In [ ]:
# TODO
X_pca = _

In [ ]:
# Dica: Transforme as variedades categorias em valores numéricos -> LabelEncoder (scklearn)
labels = _

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='rainbow', alpha=0.8, edgecolor='k')
plt.xlabel('Primeiro componente principal')
plt.ylabel('Segundo componente principal')
plt.grid()
plt.show()

In [ ]:
# TODO
# Verifique a variância explicada por cada componente.
# Podemos reduzir a dimensionalidade do nosso problema?

#### Elo7: Motivo de compra

In [ ]:
df_reason = pd.read_csv(os.path.join(DATASET_FOLDER, 'purchase_reason_elo7_dataset.csv'), sep=';')

df_reason.head(10)

In [ ]:
tfidf = TfidfVectorizer(max_df=0.9, max_features=500, sublinear_tf=True, use_idf=True)

In [ ]:
X = tfidf.fit_transform(df_reason['reason'].values)

O TF-IDF cria uma representação esparsa para a matriz de atributos. Isso se deve ao fato de que existem muito mais "zeros" do que números na nossa matriz. Para isso, precisamos usar a função [`TruncatedSVD`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) do sklearn.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=X.shape[1]-1)

In [ ]:
X_svd = svd.fit_transform(X)

Encontrar a variância explicada pelos componentes principais.

In [ ]:
# Componentes principais
features = range(svd.components_.shape[1]-1)

# Ordena os componentes - diferente do PCA que já vem ordenado
exp_var = np.sort(svd.explained_variance_ratio_)[::-1]

# Apresenta apenas os 50 primeiros
plt.bar(features[:50], exp_var[:50])
plt.xticks(features[:50])
plt.xlabel('Componentes principais')
plt.ylabel('Variância')
plt.show()

In [ ]:
print('Variância explicada: ')
cum_exp_var = np.cumsum(exp_var)[:100]

print('-'*50)
for i in range(0,len(exp_var[:100]),10):
    print('{} Componente(s) principal(is): {:.2%}'.format(i+1, cum_exp_var[i]))

Podemos verificar que conseguiríamos reduzir consideravelmente a dimensão do problema! 

Vamos plotar o gráfico dos primeiros componentes.

In [ ]:
plt.scatter(X_svd[:, 0], X_svd[:, 1], alpha=0.8, edgecolor='k')
plt.xlabel('Primeiro componente principal')
plt.ylabel('Segundo componente principal')
plt.grid()
plt.show()

Podemos plotar também os centroides dos clusters encontrados utilizando o K-Means.

In [ ]:
# Utiliza o K-Means para encontrar os clusters
kmeans = KMeans(n_clusters=20)
kmeans.fit(X)
labels = kmeans.predict(X)

# Cria um dataframe com as labels
df = pd.DataFrame({'reason': df_reason['reason'], 'labels': labels})

# Escolhe um título para o cluster
## para cada cluster, escolhe aleatoriamente um motivo de compra
titles = df.groupby('labels').apply(lambda x: np.random.choice(list(x['reason'])))

In [ ]:
# Cria uma matriz representando os centroides dos clusters
X_centers_svd = svd.transform(kmeans.cluster_centers_)

In [ ]:
# plota os centroides dos clusters
plt.scatter(X_centers_svd[:, 0], 
            X_centers_svd[:, 1], 
            c=range(X_centers_svd.shape[0]), 
            cmap='rainbow', 
            alpha=0.8, 
            edgecolor='k')
plt.xlabel('Primeiro componente principal')
plt.ylabel('Segundo componente principal')
plt.grid()

# Coloca nomes nos pontos
for i, center in enumerate(kmeans.cluster_centers_):
    plt.annotate(titles[i], 
                 ((X_centers_svd[i, 0], X_centers_svd[i, 1])),
                 va="bottom", ha="left", rotation=30)
plt.show()

Agora estamos próximos do que gostaríamos! Em um gráfico já conseguimos ver mais informações sobre nossos dados de alta dimensionalidade!

---
# T-SNE

[T-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) é um algoritmo de machine learning iterativo que procura mapear regiões multidimensionais em 2 ou 3 dimensões. O resultado gerado pelo método é um mapa de dispersão na qual os pontos com maior afinidade (mesma estrutura) estão próximos, enquanto os pontos dissimilares ficam distantes. Note que os eixos não têm necessariamente um significado predeterminado. Não podemos calcular distâncias entre pontos a partir do resultado do T-SNE.

Normalmente, o T-SNE é utilizado para visualização de dados. Podemos mapear problemas de alta dimensionalidade em 2 ou 3 dimensões, onde garantimos que a sua estrutura se manterá.

In [ ]:
# Vamos agora importar o método T-SNE do scikit-learn
from sklearn.manifold import TSNE

#### Dataset variedade de grãos de trigo

In [ ]:
X = df_grain.drop('varieties',1).values

X_scaled = normalizer.fit_transform(X)

# Codifica variedades em valores numéricos
label_encoder = LabelEncoder()
varieties = label_encoder.fit_transform(df_grain['varieties'].values)

In [ ]:
# Execute o T-SNE
# Crie a instância do T-SNE
# Teste diferentes valores de learning_rate e perplexity
tsne = TSNE(learning_rate=200, perplexity=30)

# Transforme os dados
X_tsne = tsne.fit_transform(X)

# Agora basta plotar o resultado
plt.scatter(X_tsne[:,0], 
            X_tsne[:,1], 
            c=varieties, 
            cmap='rainbow')
plt.show()

### Artigos do Wikipedia

O dataset que vamos utilizar foi retirado desse [link](https://blog.lateral.io/2015/06/the-unknown-perils-of-mining-wikipedia/). O dataset original contém 463 mil páginas do Wikipedia de diversos temas. Para não ficarmos o dia inteiro processando os dados, foram retirados apenas 58 artigos de classes bem definidas (por exemplo, futebol, música, internet etc). Vamos analisar o dataset:

In [ ]:
df_wiki = pd.read_csv(os.path.join(DATASET_FOLDER, 'wikipedia_dataset.csv'), sep=';')

df_wiki.head()

A coluna de clusters foi feita de maneira a facilitar nosso entendimento sobre o resultado final do algoritmo. Na maioria dos casos nós não teremos esse entendimento antes de realizar uma primeira análise (clustering) dos algoritmos.

Vamos tentar aplicar os métodos PCA para reduzir a dimensionalidade desse problema e o T-SNE para visualização.

Mas, primeiro, precisamos criar a matriz de atributos. Para isso, vamos utilizar o TF-IDF.

In [ ]:
tfidf = TfidfVectorizer(max_df=0.8, max_features=15000, sublinear_tf=True, use_idf=True)

In [ ]:
# Aplique a transformação nos artigos
X = tfidf.fit_transform(df_wiki['artigo'].values)

# Tamanho do dataset
X.shape

Como temos 15000 dimensões, nós podemos ter problemas com a tal da *maldição da dimensionalidade*. Além disso, a probabilidade de cada uma das palavras estar dentro de todos os documentos é muito baixa. Ou seja, a matriz de atributos é muito esparsa. Isso nos dá um indício forte para acreditar que teremos grandes vantagens se utilizarmos uma técnica para reduzir a dimensionalidade, como o PCA.

In [ ]:
# Vamos utilizar o TruncatedSVD para reduzir a dimensionalidade do dataset.
# Podemos limitar o número de components para ser igual ao número
# de dados: 58
# Desse modo temos uma matriz quadrada
svd = TruncatedSVD(n_components=58)

# Aplique o método SVD
X_svd = svd.fit_transform(X)

# Vamos ver como ficou o novo tamanho
X_svd.shape

É uma redução bastante drástica! Reduzimos de 15000 dimensões para apenas 58. Será que perdemos muita informação? Vamos verificar o gráfico de variância explicada.

In [ ]:
# Componentes principais
features = range(svd.components_.shape[1]-1)

# Ordena os componentes - diferente do PCA que já vem ordenado
exp_var = np.sort(svd.explained_variance_ratio_)[::-1]

# Apresenta apenas os 50 primeiros
plt.bar(features[:50], exp_var[:50])
plt.xticks(features[:50])
plt.xlabel('Componentes principais')
plt.ylabel('Variância')
plt.show()

In [ ]:
print('Variância explicada: ')
cum_exp_var = np.cumsum(exp_var)

print('-'*50)
for i in range(0,len(exp_var),10):
    print('{} Componente(s) principal(is): {:.2%}'.format(i+1, cum_exp_var[i]))

Podemos ver que ao utilizar 50 componentes, nós já teríamos aproximadamente 90% da variância explicada. Isso nos mostra que poderíamos reduzir ainda mais o número de dimensões escolhidas.

Vamos agora plotar o gráfico T-SNE para verificar a estrutura dos dados.

In [ ]:
tsne = TSNE(learning_rate=5)

X_tsne = tsne.fit_transform(X_svd)

In [ ]:
labels = LabelEncoder().fit_transform(df_wiki['cluster'].values)

plt.scatter(X_tsne[:, 0],
            X_tsne[:, 1],
            c=labels,
            cmap='rainbow')

for i in range(X_tsne.shape[0]):
    # Anota os motivos no gráfico
    plt.annotate(df_wiki['titulo'][i], 
                 ((X_tsne[i, 0], X_tsne[i, 1])),
                 xytext=(4, 3), 
                 textcoords='offset points')
plt.grid()
plt.show()

Nós conseguímos apresentar dados de 15000 dimensões em apenas duas e, ainda assim, verificar a presença de uma estrutura bem definida entre os clusters. Incrível, não?

### Case Elo7: Motivos de compra

Agora tente aplicar os métodos PCA e T-SNE para encontrar relações entre os dados. Utilize as ferramentas apresentadas anteriormente para gerar um bom entendimento sobre o problema.

In [ ]:
X = tfidf.fit_transform(df_reason['reason'].values)

Aplique os métodos PCA e T-SNE.

In [ ]:
# TODO
# 
X_svd = 

In [2]:
# TODO
# Encontre a quantidade de dimensões necessárias para o nosso problema

In [ ]:
# TODO: Coloque o número de componentes escolhido anteriormente
svd = TruncatedSVD(n_components=___)

# Aplique o método SVD em um sample dos dados
df_sample = df_reason.sample(n=75)
X_sample = tfidf.transform(df_sample['reason'].values)

X_svd = svd.fit_transform(X_sample)

In [ ]:
# TODO
# Execute o T-SNE
# Teste diferentes valores de learning_rate e perplexity
tsne = TSNE(___)

X_tsne = _

Como fizemos para o PCA (ou TruncatedSVD), podemos plotar os clusters obtidos pelo K-Means.

In [ ]:
# Apenas para auxiliar a verificação dos clusters
kmeans = KMeans(n_clusters=10)
kmeans.fit(X_svd)
labels = kmeans.predict(X_svd)

# Cria um dataframe com as labels
df = pd.DataFrame({'reason': df_sample['reason'], 'labels': labels})

In [ ]:
# Apresenta o resultado do T-SNE de duas dimensões com as cores
# indicando cada um dos clusters pré-definidos
labels_sample = df['labels'].values
titles_sample = df['reason'].values

plt.scatter(X_tsne[:, 0],
            X_tsne[:, 1],
            c=labels_sample,
            cmap='rainbow')

for i in range(X_tsne.shape[0]):
    # Anota os motivos no gráfico
    plt.annotate(titles_sample[i][:100], 
                 ((X_tsne[i, 0], X_tsne[i, 1])),
                 xytext=(4, 3), 
                 textcoords='offset points')
plt.grid()
plt.show()

# Topic Analysis

Até agora, nós utilizamos diversos algoritmos de clustering e redução de dimensionalidade para conseguir encontrar relações de proximidade entre documentos. Entretanto, apesar de algoritmos como o PCA conseguirem representar reduzidamente o nosso conjunto de documentos, nós não conseguíamos interpretar o resultado obtido. Isso acontece porque o PCA encontra novos vetores de features que são combinações lineares do conjunto de palavras existentes. Esse fator pode não ser um problema se o que se deseja é apenas encontrar clusters sem interpretações mais profundas. Entretanto, muitas vezes gostaríamos de entender o racional por trás da geração dos clusters. Ainda mais, as vezes gostaríamos de reduzir um documento a um conjunto de palavras-chave que podem "resumir" o nosso documento e agrupá-las em **tópicos**. E é exatamente esse o objetivo dessa aula.

A área de topic analysis é de grande importância para Machine Learning, ou mais especificamente a área de Data Mining. A utilização de tópicos nos permite ter uma melhor e mais compacta representação dos nossos dados, principalmente quando temos um conjunto extenso de dados e atributos (features). 

Utilizar topic analysis em Natural Language Processing (NLP) é algo bem intuitivo. Nós naturalmente fazemos isso quando queremos organizar textos (documentos) em diferentes categorias, ou temas. Por exemplo, nós podemos ler artigos do Google News e dizer facilmente que um determinado artigo tem o tema "esporte", ou o tema "política". Nosso trabalho em topic analysis é o de conseguir desenvolver algoritmos de Machine Learning que possam encontrar automaticamente esses tópicos, ou temas, por nós.

## Non-Negative Matrix Factorization (NMF)

O [NMF](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization) é um algoritmo poderoso (apesar de relativamente simples) para encontrar tópicos em um conjunto de documentos e features. Ele se baseia em um processo de decomposição de matrizes para criar uma representação adequada da matriz de frequência de palavras (denotado por **A**). Mais especificamente, o NMF decompõe a matriz de frequência de palavras em duas: a primeira é a matriz de pesos (chamada de **W**), com as linhas representando os documentos e as colunas indicando os tópicos; e a segunda é a matriz de atributos (chamada de **H**), com as linhas indicando os tópicos e as colunas os atributos. O número de tópicos é definido antecipadamente e é fixo.

<img src="./imagens/nmf_draw.png" alt="Drawing" style="width: 500px;"/>

As duas matrizes são formadas a partir de um processo iterativo de otimização (veja esse [link](http://www.columbia.edu/~jwp2128/Teaching/E4903/papers/nmf_nature.pdf) para mais detalhes) com o objetivo de reconstruir fielmente a matriz **A**. Entretanto, para esse fim, a matriz **A** não pode possuir entradas negativas.

Vamos aplicar esse método na prática para verificar o resultado!

### Exemplo Wikipedia

In [ ]:
df_wiki = pd.read_csv(os.path.join(DATASET_FOLDER, 'wikipedia_dataset.csv'), sep=';')

df_wiki.head()

In [ ]:
tfidf = TfidfVectorizer(max_df=0.8, max_features=15000, sublinear_tf=True, use_idf=True)

X = tfidf.fit_transform(df_wiki['artigo'].values)

In [ ]:
# Primeiro, importe o módulo NMF do scikit-learn
from sklearn.decomposition import NMF

# Precisamos criar a instância do NMF
# Temos que definir um número de componentes para o NMF
# Como temos 6 clusters, vamos escolher n_components=6
nmf = NMF(n_components=6)

# Agora vamos utilizar os mesmos atributos fit, transform ou fit_transform
# que já conhecemos do universo do sklearn
W_nmf = nmf.fit_transform(X)

# Vamos ver qual é a dimensão de W_nmf
W_nmf.shape

Note que o número de linhas se manteve em 58, que é o número de documentos (artigos) que nós temos, e o número de colunas se transformou em 6, que é o número de tópicos que nós escolhemos. Essa matriz gerada representa a matriz **W** (matriz de pesos) da fatoração de matrizes.

Vamos agora achar a matriz **H** que representa a matriz de atributos.

In [ ]:
H_nmf = nmf.components_
H_nmf.shape

Note que o número de linhas é igual ao número de tópicos e o número de colunas representa o número de palavras no nosso vocabulário. Cada linha da matriz é definida como um componente (assim como o PCA possui os componentes principais) que está associado a um tópico específico. Entretanto, diferentemente do PCA, nós podemos associar cada componente a um conjunto específico de palavras. Vamos verificar abaixo:

In [ ]:
# Precisamos criar uma lista de palavras que representam as 
# colunas da matriz de frequência de palavras
words = [x[0] for x in sorted(tfidf.vocabulary_.items())]

# Vamos criar um dataframe para visualizar
components_df = pd.DataFrame(nmf.components_, columns=words)

# Vamos verificar as palavras que representam cada um dos tópicos
for i in range(6):
    component = components_df.iloc[i]
    print("Topico {}:".format(i))
    print("----------")
    print(component.nlargest(10))
    print("----------")

O que achou da distribuição de palavras dentro de cada tópico? Acha que faz sentido com os temas principais dos artigos do Wikipedia? Cada um dos tópicos poderia ser associado a um cluster?

Podemos ainda verificar quais são os tópicos principais de alguns artigos específicos.

In [ ]:
# Precisamos criar um dataframe para facilitar nossa vida
df = pd.DataFrame(W_nmf, index=df_wiki['titulo'])

examples = ['Denzel Washington','Leukemia','Neymar','LinkedIn','Arctic Monkeys','Global warming']

for ex in examples:
    print('Artigo: {}'.format(ex))
    print('-'*30)
    print('Topico\tDist.\tPalavras')
    print('-'*30)
    for i,topic in enumerate(df.loc[ex]):
        words = components_df.iloc[i].nlargest(5).index.tolist()
        print('{}\t{:.2f}\t{}'.format(i,topic,', '.join(words)))
    print()

Podemos notar que os artigos possuem tópicos coerentes com o que esperávamos!

Vamos agrupar agora os artigos pelos tópicos principais de cada um deles e ver como eles tém relação com os clusters definidos anteriormente.

In [ ]:
# Cria as labels a partir do tópico mais relevante de cada artigo
labels = np.argmax(W_nmf, axis=1)

# Cria o novo dataframe com os labels dos clusters
df = pd.DataFrame({'label': labels, 'article': df_wiki['titulo']})

# Apresenta os resultados
print(df.sort_values(by='label'))

Só por diversão, vamos verificar como ficaria um agrupamento usando Hierarchical Clustering na matriz de atributos obtida pelo NMF.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

Y = linkage(W_nmf, method='ward', metric='euclidean')

dendrogram(Y,
           labels=df_wiki['titulo'].values,
           leaf_rotation=90,
           leaf_font_size=14,
)
plt.show()

O que achou do resultado? Percebeu que o NMF não só encontrou uma representação em tópicos dos documentos, mas também teve um papel de agregador? Ele realizou um ótimo trabalho em encontrar clusters! E o melhor, nós podemos explicar com palavras o que representa cada um dos tópicos/clusters.

### Case Elo7: Motivos de Compra

Agora aplique o NMF no dataset de motivos de compra e verifique o resultado comparado ao PCA.

In [ ]:
df_reason.head()

In [ ]:
tfidf = TfidfVectorizer(max_df=0.8, max_features=5000, sublinear_tf=True, use_idf=True)

X = tfidf.fit_transform(df_reason['reason'].values)

In [ ]:
# TODO
# Encontre a matriz de pesos do NMF
W_nmf = _

In [ ]:
# Precisamos criar uma lista de palavras que representam as 
# colunas da matriz de frequência de palavras
words = [x[0] for x in sorted(tfidf.vocabulary_.items())]

# Vamos criar um dataframe para visualizar
components_df = pd.DataFrame(nmf.components_, columns=words)

# Vamos verificar as palavras que representam cada um dos tópicos
for i in range(W_nmf.shape[1]):
    component = components_df.iloc[i]
    print("Topico {}:".format(i))
    print("----------")
    print(component.nlargest(10))
    print("----------")

In [ ]:
# Cria as labels a partir do tópico mais relevante de cada artigo
labels = np.argmax(W_nmf, axis=1)

# Cria o novo dataframe com os labels dos clusters
df = pd.DataFrame({'label': labels, 'reason': df_reason['reason']})

# Pega um sample
df_sample = df.sample(n=60)

In [ ]:
# Vamos verificar a relação entre cada tópico encontrado
# Para isso, vamos utilizar o hierarchical clustering
X_sample = df_sample['reason'].values

W_nmf = nmf.fit_transform(tfidf.fit_transform(X_sample))

Y = linkage(W_nmf, method='ward', metric='euclidean')

dendrogram(Y,
           labels=df_sample['reason'].values,
           leaf_rotation=90,
           leaf_font_size=14,
)
plt.show()

## Latent Dirichlet Allocation (LDA)

Assim como o NMF, o [LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) é um algoritmo relativamente simples (pelo menos algoritmicamente) e poderoso para conseguir encontrar tópicos dentro de documentos. Entretanto, diferentemente do NMF, o LDA se baseia em métodos probabilísticos. Mais especificamente, o **LDA** assume que cada **documento** foi gerado a partir de uma **mistura de tópicos** com uma distribuição de probabilidade (no caso, uma distribuição [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution)), e cada **tópico** foi gerado por uma distribuição de **palavras** com certa probabilidade associada (distribuição [multinomial](https://en.wikipedia.org/wiki/Multinomial_distribution)).

Para imaginar um cenário, imagine que temos os seguintes documentos (retirados deste [link](https://www.quora.com/What-is-a-good-explanation-of-Latent-Dirichlet-Allocation)):
1. I ate a banana and spinach smoothie for breakfast
2. I like to eat broccoli and bananas.
3. Chinchillas and kittens are cute.
4. My sister adopted a kitten yesterday.
5. Look at this cute hamster munching on a piece of broccoli.

Para o LDA, os documentos poderiam ser formados pelos seguintes tópicos e palavras:
- Documentos 1 e 2: 100% Tópico A
- Documentos 3 e 4: 100% Tópico B
- Documento 5: 60% Tópico A, 40% Tópico B

- Tópico A: 30% broccoli, 15% bananas, 10% breakfast, 10% munching (Talvez represente algo relacionado a comida)
- Tópico B: 20% chinchillas, 20% kittens, 20% cute, 15% hamster (Talvez seja relacionado a animais fofinhos)

Esse comportamento do LDA é interessante, já que temos uma representação mais intuitiva da formação dos documentos. Poderíamos até criar documentos artificiais a partir dessas definições.

Agora vamos ver um exemplo de aplicação utilizando o dataset do Wikipedia.

In [ ]:
df_wiki = pd.read_csv(os.path.join(DATASET_FOLDER, 'wikipedia_dataset.csv'), sep=';')

df_wiki.head()

O LDA não pode ser utilizado com Tf-Idf, porque ele precisa da contagem total de palavras nos documentos. Por esse motivo, vamos utilizar o método [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) do scikit-learn. Esse método apenas realiza a contagem de frequência de palavras nos documentos. Para evitar que "stopwords" tenham uma importância indevida, podemos remove-las do texto.

In [ ]:
# Importa o módulo CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tf = CountVectorizer(max_df=0.95, min_df=0.01, max_features=15000, stop_words='english')

X = tf.fit_transform(df_wiki['artigo'].values)

In [ ]:
# Agora vamos importar o módulo LDA do scikit-learn
from sklearn.decomposition import LatentDirichletAllocation

# Precisamos criar a instância do LDA
# Temos que definir um número de tópicos do LDA
# Como temos 6 clusters, vamos escolher n_topics=6
lda = LatentDirichletAllocation(n_topics=6, learning_method='batch')

# Agora vamos utilizar os mesmos atributos fit, transform ou fit_transform
# que já conhecemos do universo do sklearn
X_lda = lda.fit_transform(X)

# Vamos ver qual é a dimensão de X_lda
X_lda.shape

Note que o número de linhas se manteve em 58, que é o número de documentos (artigos) que nós temos, e o número de colunas se transformou em 6, que é o número de tópicos que nós escolhemos. 

Vamos agora achar a matriz de atributos.

In [ ]:
# Normaliza os componentes
lda_components = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
print(lda_components.shape)

Note que o número de linhas é igual ao número de tópicos e o número de colunas representa o número de palavras no nosso vocabulário. Essa matriz é semelhante a matriz de atributos do NMF.

Vamos achar quais são as palavras mais importantes para cada tópico.

In [ ]:
# Precisamos criar uma lista de palavras que representam as 
# colunas da matriz de frequência de palavras
words = [x[0] for x in sorted(tf.vocabulary_.items())]

# Vamos criar um dataframe para visualizar
components_df = pd.DataFrame(lda_components, columns=words)

# Vamos verificar as palavras que representam cada um dos tópicos
for i in range(6):
    component = components_df.iloc[i]
    print("Topico {}:".format(i))
    print("----------")
    print(component.nlargest())
    print("----------")

O que achou da distribuição de palavras dentro de cada tópico? Acha que faz sentido com os temas principais dos artigos do Wikipedia? Cada um dos tópicos poderia ser associado a um cluster?

Podemos ainda verificar quais são os tópicos principais de alguns artigos específicos.

In [ ]:
# Precisamos criar um dataframe para facilitar nossa vida
df = pd.DataFrame(X_lda, index=df_wiki['titulo'])

examples = ['Denzel Washington','Leukemia','Neymar','LinkedIn','Arctic Monkeys','Global warming']

for ex in examples:
    print('Artigo: {}'.format(ex))
    print('-'*30)
    print('Topico\tDist.\tPalavras')
    print('-'*30)
    for i,topic in enumerate(df.loc[ex]):
        words = components_df.iloc[i].nlargest(5).index.tolist()
        print('{}\t{:.2f}\t{}'.format(i,topic,', '.join(words)))
    print()

In [ ]:
# Cria as labels a partir do tópico mais relevante de cada artigo
labels = np.argmax(X_lda, axis=1)

# Cria o novo dataframe com os labels dos clusters
df = pd.DataFrame({'label': labels, 'titulo': df_wiki['titulo']})

In [ ]:
Y = linkage(X_lda, method='ward', metric='euclidean')

dendrogram(Y,
           labels=df_wiki['titulo'].values,
           leaf_rotation=90,
           leaf_font_size=14,
)
plt.show()

---
# Case Elo7 - Subcategorias Automáticas

Vamos para mais um case real do Elo7!

Esse case é um dos trabalhos mais recentes do time de Data Science do Elo7. De fato, é um trabalho ainda em aberto e qualquer sugestão de melhorias é bem vinda! =)

- O problema:
O Elo7 possui uma árvore de categorias dividida em N1 e N2. O primeiro nível (N1) contém as categorias "alto nível" do site. São as categorias mais genéricas do marketplace- ou, pelo menos, é assim gostaríamos que fosse. As categorias N2, ou subcategorias, são as possíveis extensões dos nós das categorias N1. Podemos perceber que a árvore é extremamente limitada e isso é um problema grave não só para os compradores, que não conseguem navegar nas nossas categorias, mas também para os vendedores, que não conseguem categorizar bem seus produtos. A solução para esse problema seria uma árvore de categorias com maior "granularidade", ou seja, que consiga expandir além dos 2 níveis e ter mais subcategorias.

- O que o time de Data Science tem a ver com essa história? 

Bom, gerar uma nova árvore de categoria pode ser uma tarefa bastante monótona e cansativa. Provavelmente deve haver algum jeito de encontrar bons agrupamentos de produtos que pudessem servir como uma nova subcategoria. Talvez algum método de clustering que utilize como features o conteúdo dos produtos pode gerar algum resultado interessante.

- O experimento:

O dataset a seguir possui um subconjunto de produtos que foram categorizados na categoria N1 "Casamento". Escolhemos esse conjunto de dados para iniciar nossos trabalhos, porque assim temos mais controle sobre nossos resultados. E, também, porque é uma das categorias mais importantes do marketplace.

Para essa tarefa, vamos utilizar apenas o título e uma parte da descrição do produto (aprox. 140 caracteres) como features de entrada.

Vamos analisar os dados!

In [ ]:
df_cat = pd.read_csv(os.path.join(DATASET_FOLDER, 'subcategory_elo7_dataset.csv'), sep=';')

df_cat.head(10)

Vamos criar uma coluna com as features que vamos incluir no nosso modelo de aprendizagem.
Esse vetor de features será o título + descrição do produto. Para compensar a quantidade de palavras do título em relação a descrição, vamos repetir o título duas vezes.

In [ ]:
df_cat['title_desc'] = (df_cat['title'] + ' ')*2 + df_cat['short_description']

df_cat.head(10)

Para criar nossa matriz de features, nós vamos utilizar o Tf-Idf.

In [ ]:
tfidf = TfidfVectorizer(max_df=0.9, max_features=10000, sublinear_tf=True, use_idf=True)

In [ ]:
X = tfidf.fit_transform(df_cat['title_desc'].values)

Agora aplique os métodos encontrados até agora para tentar encontrar uma boa árvore de categorias.

In [ ]:
# TODO

---
# Case Sistema de Recomendação

Esse case foi retirado desse [artigo](https://towardsdatascience.com/topic-modeling-for-the-new-york-times-news-dataset-1f643e15caac). O objetivo dele é verificar se conseguimos extrair tópicos relevantes de um dataset contendo 8.447 matérias do NY Times, com um vocabulário de 3.012 palavras. Por razão de direitos autorias, os documentos não possuem título.

<img src="https://cdn-images-1.medium.com/max/1400/1*toWf7lAVf_5GIb9IMfS8Bw.jpeg" alt="Drawing" style="width: 800px;"/>


O arquivo `nyt_dataset.csv` contém os documentos, onde cada linha representa um documento específico e cada coluna representa a frequência de cada palavra. O dataset já sofreu um processo de limpeza e vetorização, onde foram mantidas apenas palavras que ocorreram mais de 10 vezes.

Vamos criar a matriz de frequências a partir desses dados:

In [ ]:
df_nyt = pd.read_csv(os.path.join(DATASET_FOLDER, 'nyt_dataset.csv'), sep=';')

df_nyt.head()

In [ ]:
X = df_nyt.values

In [ ]:
# TODO
# Encontrar a matriz de pesos do NMF
# Dica: escolher entre 20 e 25 tópicos
# Teste com outros números e verifique 
# a qualidade

W_nmf = _

Note que o número de linhas se manteve em 8447, que é o número de documentos (artigos) que nós temos, e o número de colunas se transformou em 25, que é o número de tópicos que nós escolhemos. Essa matriz gerada representa a matriz **W** (matriz de pesos) da fatoração de matrizes.

Vamos agora achar a matriz **H** que representa a matriz de atributos.

In [ ]:
# TODO

Foi possível verificar que o número de linhas é igual ao número de tópicos e o número de colunas representa o número de palavras no nosso vocabulário? Cada linha da matriz é definida como um componente (assim como o PCA possui os componentes principais) que está associado a um tópico específico. Entretanto, diferentemente do PCA, nós podemos associar cada componente a um conjunto específico de palavras. Vamos verificar abaixo:

In [ ]:
# TODO
# Verifique a distribuição de palavras em cada topico

#### Sistemas de recomendação

Em poucos anos a internet revolucionou o mercado de consumo mundial e a forma como os clientes interagem com os vendedores. Uma das dinâmicas criadas mais importantes a partir dessa revolução é a de sistemas de recomendação. Todos devem já ter notado o quanto e-commerces como a Amazon, Best Buy e até empresas brasileiras acertam ao recomendar certos tipos de produto para seus clientes, que muitas vezes nem os estavam procurando (veja esse [artigo](https://www.techemergence.com/use-cases-recommendation-systems/)). Isso não acontece mais apenas em e-commerces, mas também com provedores de música (Spotify), de filmes (Netflix) ou vídeos em geral (Youtube). 

Esse fenômeno só ocorre devido a um fator: **dados**. As empresas atualmente possuem muita informação sobre os seus produtos e os seus usuários. É muito fácil obter, hoje em dia, os interesses dos clientes sobre seus produtos. Seja o fato de o vendedor comprar um produto, dar um review ou apenas clicar, já é suficiente para uma empresa mapear os interesses dos usuários e tentar direcionar produtos que seriam relevantes para o usuário sem nem mesmo ele saber!

Os sistemas de recomendação se baseiam, basicamente, em encontrar relações entre compradores e produtos. Mais especificamente, existem dois grandes grupos de sistemas de recomendação:
- **Proximidade de produtos**: Tem o objetivo de encontrar produtos similares aos consumidos por um cliente. Se um cliente possui o interesse em um determinado produto, o sistema de recomendação pode tentar encontrar outros produtos similares para indicar para o cliente.
- **Proximidade entre clientes** (Filtro Colaborativo): Tem o objetivo de encontrar clientes com interesses semelhantes. Suponha que exista um cliente X que consuma os produtos A e B, enquanto um outro cliente Y tem interesse nos produtos A, B e C. Como eles possuem interesses semelhantes (produtos A e B), o sistema de recomendação poderia indicar o produto C para o cliente X.

Existem diversos outros tipos de sistemas de recomendação que fogem do escopo desse material. Mais informações podem ser vistas nesse [link](https://www.techemergence.com/use-cases-recommendation-systems/).

##### Exemplo
Agora vamos tentar criar um sistema simples de recomendação baseado em **proximidade de produtos**! 

Vamos utilizar o dataset de artigos do NY Times para essa tarefa. Vamos supor que uma pessoa tenha lido um determinado artigo e nós gostaríamos de recomendar outros artigos semelhantes àquele. Lembre que para comparar dois documentos que contém vetores de atributos associados a palavras, a melhor medida de distância é a **distância de cossenos**.

Para calcular a distância entre produtos, nós poderíamos utilizar todo o espaço de atributos (quantidade de palavras no vocabulário), mas isso é desnecessário. Nós temos uma representação resumida de cada documento por sua proporção de tópicos, que formam o novo vetor de atributos. Como já encontramos os tópicos relacionados aos documentos no exercício anterior, podemos aproveitá-los para resolver nosso problema atual.

In [ ]:
# Nós vamos ter que normalizar o vetor de atributos
# Isso é necessário para que todas as features (tópicos) 
# de um documento some 1 ao final
# Seria a porcentagem de composição dos tópicos 
# no documento
from sklearn.preprocessing import normalize
W_nmf_norm = normalize(W_nmf)

In [ ]:
# Vamos criar um dataframe:
# índice: documento
# colunas: vetor de features (normalizadas)
df = pd.DataFrame(W_nmf_norm,
                  columns=['topic {}'.format(i) for i in range(n_topics)],
                  index=['doc {}'.format(i) for i in range(len(df_nyt))])
df.head(5)

In [ ]:
# Vamos escolher um documento aleatoriamente
# Esse documento será o escolhido pelo cliente
doc_target = np.random.choice(range(len(df_nyt)))
doc_target

In [ ]:
# Verifica o vetor de features dele:
doc_target_features = df.iloc[doc_target]
print(doc_target_features)

In [ ]:
# Calcula a distância de cossenos entre o artigo e 
# todos os outros documentos
cossine_distance = df.dot(doc_target_features)

# Todos os documentos com maior distância de cossenos 
# representam os documentos mais próximos
print("Artigos recomendados:")
recommendations = cossine_distance.nlargest(6)[1:]
print(recommendations)

In [ ]:
default_top_topics = df.iloc[doc_target].nlargest(3).index.values
default_components = [components_df.iloc[int(i.split(' ')[1])].nlargest(5).index.values 
                      for i in default_top_topics]

print("Artigo original: doc {}".format(doc_target))
print("-----------------------")
print("- Tópicos (palavras): ")
for i in range(len(default_top_topics)):
    print("{:>15} ({})".format(default_top_topics[i], ', '.join(default_components[i])))
print("\n\n")

for doc, similarity in recommendations.items():
    doc_num = int(doc.split(' ')[1])
    top_topics = df.iloc[doc_num].nlargest(3).index.values
    components = [components_df.iloc[int(i.split(' ')[1])].nlargest(5).index.values for i in top_topics]
    print("Recomendação {}:".format(doc))
    print("-----------------------")
    print("- Similaridade: {:.2%}".format(similarity))
    print("- Tópicos (palavras): ")
    for i in range(len(top_topics)):
        print("{:>15} ({})".format(top_topics[i], ', '.join(components[i])))
    print("\n")